In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km

    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c

In [11]:
gnss="stations_tohoku_bbox.csv"
seismic="processed_pgv_data2.csv"

gnss_df=pd.read_csv(gnss)
seismic_df=pd.read_csv(seismic)

거리 제한 O

In [17]:
pairs=[]
distance=20

for g_idx, g_row in gnss_df.iterrows():
    min_dist = float('inf')
    closest_s = None

    for s_idx, s_row in seismic_df.iterrows():
        dist = haversine(
            s_row['latitude'],
            s_row['longitude'],
            g_row['latitude'],
            g_row['longitude']
        )

        if dist < min_dist:
            min_dist = dist
            closest_s = s_row

    if min_dist <= distance and closest_s is not None:
        pairs.append({
            'seismic_station': closest_s['station'],
            'gnss_station': g_row['station'],
            'seismic_lat': closest_s['latitude'],
            'seismic_lon': closest_s['longitude'],
            'gnss_lat': g_row['latitude'],
            'gnss_lon': g_row['longitude'],
            'pgv': closest_s['pgv'],
            'distance_km': min_dist
        })

pairs_df = pd.DataFrame(pairs)

pairs_df.to_csv('station_pairs_ver_20km.csv', index=False)

print(f"20km 이내의 관측소 쌍: {len(pairs_df)}개")
print(pairs_df.head())

20km 이내의 관측소 쌍: 176개
  seismic_station gnss_station  seismic_lat  seismic_lon   gnss_lat  \
0          N.AJSH     GNET0026      40.7618     140.3121  40.778983   
1          N.KJNH     GNET0027      40.2686     141.7831  40.133444   
2          N.YMDH     GNET0028      39.4734     141.9336  39.572374   
3          N.SMTH     GNET0029      39.1809     141.3909  39.110614   
4          N.TDOH     GNET0035      38.3841     140.3805  38.331024   

     gnss_lon      pgv  distance_km  
0  140.272985  11.0690     3.807970  
1  141.789102   8.9787    15.037304  
2  141.939980  16.4382    11.019003  
3  141.203945  28.2287    17.916762  
4  140.366105  20.9417     6.033775  


거리 제한 X

In [13]:
pairs=[]

for g_idx, g_row in gnss_df.iterrows():
    min_dist = float('inf')
    closest_s = None

    for s_idx, s_row in seismic_df.iterrows():
        dist = haversine(
            s_row['latitude'],
            s_row['longitude'],
            g_row['latitude'],
            g_row['longitude']
        )

        if dist < min_dist:
            min_dist = dist
            closest_s = s_row

    if closest_s is not None:
        pairs.append({
            'seismic_station': closest_s['station'],
            'gnss_station': g_row['station'],
            'seismic_lat': closest_s['latitude'],
            'seismic_lon': closest_s['longitude'],
            'gnss_lat': g_row['latitude'],
            'gnss_lon': g_row['longitude'],
            'pgv': closest_s['pgv'],
            'distance_km': min_dist
        })

pairs_df = pd.DataFrame(pairs)

pairs_df.to_csv('station_pairs.csv', index=False)

print(f"관측소 쌍: {len(pairs_df)}개")
print(pairs_df.head())

관측소 쌍: 213개
  seismic_station gnss_station  seismic_lat  seismic_lon   gnss_lat  \
0          N.SRUH     GNET0023      41.5977     140.4207  41.466375   
1          N.KUCH     GNET0024      41.2340     140.9896  41.300771   
2          N.AJSH     GNET0026      40.7618     140.3121  40.778983   
3          N.KJNH     GNET0027      40.2686     141.7831  40.133444   
4          N.YMDH     GNET0028      39.4734     141.9336  39.572374   

     gnss_lon      pgv  distance_km  
0  140.040654   9.7361    34.842255  
1  141.213277   7.2237    20.114972  
2  140.272985  11.0690     3.807970  
3  141.789102   8.9787    15.037304  
4  141.939980  16.4382    11.019003  


관측소 쌍 지도 시각화

In [18]:
import pandas as pd
import folium

# 데이터 불러오기
df = pd.read_csv('station_pairs_ver_20km.csv')

# 지도의 중심점을 데이터의 중간값으로 설정
center_lat = df['seismic_lat'].mean()
center_lon = df['seismic_lon'].mean()

# 지도 객체 생성 (배경지도)
m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

for idx, row in df.iterrows():
    # 지진파 관측소 마커 추가 (빨간색)
    folium.Marker(
        location=[row['seismic_lat'], row['seismic_lon']],
        popup=f"Seismic: {row['seismic_station']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

    # GNSS 관측소 마커 추가 (파란색)
    folium.Marker(
        location=[row['gnss_lat'], row['gnss_lon']],
        popup=f"GNSS: {row['gnss_station']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

    # 두 관측소를 잇는 선 (PolyLine) 그리기
    folium.PolyLine(
        locations=[
            [row['seismic_lat'], row['seismic_lon']],
            [row['gnss_lat'], row['gnss_lon']]
        ],
        color='purple',
        weight=2,
        opacity=0.7
    ).add_to(m)

# 지도를 HTML 파일로 저장 (웹 브라우저에서 열기 가능)
m.save('station_pairs_ver_20km_interactive_map.html')